In [ ]:
from pathlib import Path
import numpy as np
import rasterio.enums
from geotools import reproject_using_grid, georef_data_array
from products.filenames import get_datetime_from_viirs_nasa_filepath
from grids import Grid, UTM375mGrid
from pyproj import CRS
import glob
import rasterio
import xarray as xr
from datetime import timedelta

meteofrance_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/EOFR62/VIIRS2024"
nasa_geometry_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_UTM_375m"
nasa_geometry_reprojected_folder = "/home/imperatoren/work/VIIRS_S2_comparison/data/V03IMG/VNP03IMG_GEO_250m"
nasa_geometry_files = sorted(glob.glob(f"{nasa_geometry_folder}/*.nc"))

meteofrance_sample_file = glob.glob(f"{meteofrance_folder}/*.LT")[0]
dst_sample = rasterio.open(meteofrance_sample_file)
dst_shape = (dst_sample.height, dst_sample.width)

meteofrance_grid = Grid(resolution=dst_sample.transform.a,x0=dst_sample.transform.c,y0=dst_sample.transform.f,width=dst_sample.width, height=dst_sample.height, crs=dst_sample.crs)
dst_transform = dst_sample.transform
dst_crs = {'init': 'EPSG:4326'}
destination = np.zeros(dst_shape, np.uint8)



# Same as processed NASA geoemtry 
nasa_grid = UTM375mGrid()
while len(nasa_geometry_files)>0:
    print("Processing file", nasa_geometry_files[0])

    file = nasa_geometry_files[0]
    file_plus_1=nasa_geometry_files[1]
    if get_datetime_from_viirs_nasa_filepath(file_plus_1) - get_datetime_from_viirs_nasa_filepath(file) == timedelta(minutes=6):
        granule = xr.open_dataset(file)
        granule_plus_1 = xr.open_dataset(file_plus_1)
        source = xr.where(granule == 255, granule_plus_1, granule)
        nasa_geometry_files.remove(file)
        nasa_geometry_files.remove(file_plus_1)

    else:
        source = xr.open_dataset(file)
        nasa_geometry_files.remove(file)

    data_vars_reproj = {}
    for data_var_name in source:
        data_var_to_reproject =source[data_var_name].where(source[data_var_name]!=255, np.nan)
        reprojected_data_var = reproject_using_grid(dataset=georef_data_array(data_var_to_reproject,
                                                                              data_array_name=data_var_name, 
                                                                              crs=nasa_grid.crs),
                                                    output_grid=meteofrance_grid,
                                                    resampling_method=rasterio.enums.Resampling.nearest,
                                                    nodata=np.nan)
        
        
        data_vars_reproj[data_var_name] = reprojected_data_var.data_vars[data_var_name]
    reprojected = xr.Dataset(data_vars_reproj)

    reprojected.to_netcdf(f"{nasa_geometry_reprojected_folder}/{Path(file).name.replace("UTM_375m", "GEO_250m")}")

: 